# Transformer Optimization

## Imports, Inits, and Functions

In [1]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, pickle, sys, warnings, tqdm, time, torch, json, gzip
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

device = 'cuda' if torch.cuda.is_available() else 'cpu'
from transformers import pipeline, set_seed
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer
from datasets import load_dataset, load_metric

from benchmark import PerformanceBenchmark

The dataset we are using is the `CLINC150` dataset for intent detection which can be found [here](https://huggingface.co/datasets/clinc_oos). This dataset includes 22,500 in-scope queries across 150 intents and 10 domains like banking and travel, and also includes 1,200 out-of-score queries that belong to `oos` intent class.

In [2]:
clinc = load_dataset('clinc_oos', 'plus')
pipe = pipeline('text-classification', model='transformersbook/bert-base-uncased-finetuned-clinc')

Reusing dataset clinc_oos (/net/kdinxidk03/opt/NFS/huggingface_cache/datasets/clinc_oos/plus/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1)


  0%|          | 0/3 [00:00<?, ?it/s]

2022-07-07 14:24:30.911169: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /net/kdinxidk03/opt/NFS/su0/anaconda3/lib/
2022-07-07 14:24:30.911315: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-07-07 14:24:30.912021: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow wi

In [ ]:
pb = PerformanceBenchmark(pipe, clinc['test'])
perf_metrics = pb.run_benchmark(clinc['test'].features['intent'])

Computing Accuracy:   0%|          | 0/5500 [00:00<?, ?it/s]

Accuracy on test set - 0.867
